#### SECTION B

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import uproot
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
import scipy.optimize as opt
from scipy.stats import chi2

import random
import os

In [ ]:
# Define paths to data files
signal_file = "Bs2DsPi/Bs2DsPi_MCsignal.root"
background_file = "Bs2DsPi/Bs2DsPi_MCcomb.root"
data_file = "Bs2DsPi/Bs2DsPi_Data.root"

# Function to load data from ROOT files
def load_root_file(file_path, tree_name="DecayTree"):
    with uproot.open(file_path) as file:
        tree = file[tree_name]
        return tree.arrays(library="pd")

# Load data
signal_df = load_root_file(signal_file)
background_df = load_root_file(background_file)
data_df = load_root_file(data_file)

In [ ]:

# Print information about the datasets
print(f"Signal samples: {len(signal_df)}")
print(f"Background samples: {len(background_df)}")
print(f"Data samples: {len(data_df)}")

# Define features for training
features = [
    'B_PT', 'B_ETA', 'B_IPCHI2_OWNPV', 'B_FDCHI2_OWNPV', 'B_DIRA_OWNPV',
    'Ds_PT', 'Ds_IPCHI2_OWNPV', 'Ds_FDCHI2_OWNPV', 'Ds_DIRA_OWNPV',
    'h1_PT', 'h1_IPCHI2_OWNPV',
    'h2_PT', 'h2_IPCHI2_OWNPV',
    'h3_PT', 'h3_IPCHI2_OWNPV',
    'Pi_PT', 'Pi_IPCHI2_OWNPV'
]

# Check if any features are missing in our datasets
missing_in_signal = [f for f in features if f not in signal_df.columns]
missing_in_bg = [f for f in features if f not in background_df.columns]
missing_in_data = [f for f in features if f not in data_df.columns]

if missing_in_signal or missing_in_bg or missing_in_data:
    print(f"Missing features in signal: {missing_in_signal}")
    print(f"Missing features in background: {missing_in_bg}")
    print(f"Missing features in data: {missing_in_data}")
    # Adjust features list if needed
    features = [f for f in features if f in signal_df.columns and f in background_df.columns and f in data_df.columns]

# Prepare the training data
X_signal = signal_df[features]
X_background = background_df[features]

y_signal = np.ones(len(X_signal))
y_background = np.zeros(len(X_background))

X = pd.concat([X_signal, X_background])
y = np.concatenate([y_signal, y_background])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Make predictions on test set
y_pred = model.predict_proba(X_test)[:, 1]

# Plot ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, lw=2, label=f'ROC curve (area = {roc_auc:.3f})')
plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.savefig("roc_curve.png")

# Apply the model to the real data
data_predictions = model.predict_proba(data_df[features])[:, 1]
data_df['bdt_score'] = data_predictions

In [ ]:
# Train an XGBoost classifier
model = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=5,
    learning_rate=0.1,
    gamma=0,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='binary:logistic',
    random_state=42
)

model.fit(X_train, y_train)

# Get feature importances
importance = model.feature_importances_
indices = np.argsort(importance)[::-1]

In [ ]:

# Plot feature importances
plt.figure(figsize=(10, 6))
plt.title("Feature Importances")
plt.bar(range(len(features)), importance[indices], align="center")
plt.xticks(range(len(features)), [features[i] for i in indices], rotation=90)
plt.tight_layout()
plt.savefig("feature_importances.png")

In [ ]:
# Function to fit the B_M distribution
def fit_mass_distribution(df, mass_range=(5300, 5500)):
    # Create histogram of mass
    hist, bin_edges = np.histogram(df['B_M'], bins=50, range=mass_range)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    bin_width = bin_edges[1] - bin_edges[0]
    
    # Define fit functions
    def gaussian(x, amp, mu, sigma):
        return amp * np.exp(-0.5 * ((x - mu) / sigma)**2)
    
    def exponential(x, a, b):
        return a * np.exp(-b * x)
    
    def combined_model(x, amp_gauss, mu_gauss, sigma_gauss, a_exp, b_exp):
        return gaussian(x, amp_gauss, mu_gauss, sigma_gauss) + exponential(x, a_exp, b_exp)
    
    # Initial guesses
    p0 = [100, 5366.9, 15, 1000, 0.001]
    
    # Fit
    popt, pcov = opt.curve_fit(combined_model, bin_centers, hist, p0=p0)
    
    # Extract parameters
    amp_gauss, mu_gauss, sigma_gauss, a_exp, b_exp = popt
    
    # Calculate errors
    perr = np.sqrt(np.diag(pcov))
    
    # Calculate signal and background components
    signal_component = gaussian(bin_centers, amp_gauss, mu_gauss, sigma_gauss)
    background_component = exponential(bin_centers, a_exp, b_exp)
    
    # Integrate to get event counts
    total_signal = np.sum(signal_component) * bin_width
    total_background = np.sum(background_component) * bin_width
    
    # Goodness of fit (chi2/ndof)
    expected = combined_model(bin_centers, *popt)
    chi2_val = np.sum((hist - expected)**2 / expected)
    ndof = len(bin_centers) - len(popt)
    
    # Plot the fit
    plt.figure(figsize=(10, 8))
    plt.hist(df['B_M'], bins=50, range=mass_range, alpha=0.6, label='Data')
    
    x_fit = np.linspace(mass_range[0], mass_range[1], 1000)
    plt.plot(x_fit, combined_model(x_fit, *popt) * bin_width, 'r-', label='Combined Fit')
    plt.plot(x_fit, gaussian(x_fit, amp_gauss, mu_gauss, sigma_gauss) * bin_width, 'g--', label='Signal')
    plt.plot(x_fit, exponential(x_fit, a_exp, b_exp) * bin_width, 'b--', label='Background')
    
    plt.xlabel('B_M (MeV)')
    plt.ylabel('Events / ({:.1f} MeV)'.format(bin_width))
    plt.title(r'Fit to $B_s^0 \to D_s^- \pi^+$ Mass Distribution')
    plt.legend()
    plt.text(mass_range[0] + 10, 0.8 * max(hist), 
             f'Signal yield = {total_signal:.1f} ± {perr[0]:.1f}\n'
             f'$\mu$ = {mu_gauss:.2f} ± {perr[1]:.2f} MeV\n'
             f'$\sigma$ = {sigma_gauss:.2f} ± {perr[2]:.2f} MeV\n'
             f'S/B = {total_signal/total_background:.2f}\n'
             f'$\chi^2$/ndof = {chi2_val:.1f}/{ndof} = {chi2_val/ndof:.2f}')
    plt.tight_layout()
    plt.savefig("initial_mass_fit.png")
    
    return {
        'mu': mu_gauss,
        'sigma': sigma_gauss,
        'signal': total_signal,
        'background': total_background,
        'S_over_B': total_signal / total_background
    }


In [ ]:

# Perform initial fit to estimate signal and background
fit_results = fit_mass_distribution(data_df)
print("Initial fit results:", fit_results)

# Function to calculate S/sqrt(S+B) for different BDT score cuts
def calculate_fom(df, cut_value, mass_range=(5300, 5500)):
    # Apply BDT cut
    df_cut = df[df['bdt_score'] > cut_value]
    
    if len(df_cut) < 100:  # Not enough events for a reasonable fit
        return 0, 0, 0
    
    # Create histogram of mass
    hist, bin_edges = np.histogram(df_cut['B_M'], bins=30, range=mass_range)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    bin_width = bin_edges[1] - bin_edges[0]
    
    # Define fit functions
    def gaussian(x, amp, mu, sigma):
        return amp * np.exp(-0.5 * ((x - mu) / sigma)**2)
    
    def exponential(x, a, b):
        return a * np.exp(-b * x)
    
    def combined_model(x, amp_gauss, mu_gauss, sigma_gauss, a_exp, b_exp):
        return gaussian(x, amp_gauss, mu_gauss, sigma_gauss) + exponential(x, a_exp, b_exp)
    
    # Initial guesses based on previous fit
    p0 = [100, fit_results['mu'], fit_results['sigma'], 1000, 0.001]
    
    try:
        # Fit
        popt, _ = opt.curve_fit(combined_model, bin_centers, hist, p0=p0)
        
        # Extract parameters
        amp_gauss, mu_gauss, sigma_gauss, a_exp, b_exp = popt
        
        # Calculate signal and background in ±3σ region
        signal_region = (mu_gauss - 3*sigma_gauss, mu_gauss + 3*sigma_gauss)
        x_signal_region = np.linspace(signal_region[0], signal_region[1], 1000)
        bin_width_sr = (signal_region[1] - signal_region[0]) / len(x_signal_region)
        
        signal_component = gaussian(x_signal_region, amp_gauss, mu_gauss, sigma_gauss)
        background_component = exponential(x_signal_region, a_exp, b_exp)
        
        # Integrate to get event counts in signal region
        S = np.sum(signal_component) * bin_width_sr
        B = np.sum(background_component) * bin_width_sr
        
        # Calculate Figure of Merit: S/sqrt(S+B)
        fom = S / np.sqrt(S + B) if (S + B) > 0 else 0
        
        return fom, S, B
    
    except:
        return 0, 0, 0

In [ ]:

# Optimize the BDT cut
cut_values = np.linspace(0, 0.9, 20)
fom_values = []
s_values = []
b_values = []

for cut in cut_values:
    fom, s, b = calculate_fom(data_df, cut)
    fom_values.append(fom)
    s_values.append(s)
    b_values.append(b)
    print(f"Cut: {cut:.2f}, FOM: {fom:.2f}, S: {s:.1f}, B: {b:.1f}, S/B: {s/b if b > 0 else 0:.2f}")

# Find the optimal cut
best_idx = np.argmax(fom_values)
best_cut = cut_values[best_idx]
best_fom = fom_values[best_idx]
best_s = s_values[best_idx]
best_b = b_values[best_idx]

# Plot FOM vs. cut value
plt.figure(figsize=(10, 6))
plt.plot(cut_values, fom_values, 'o-')
plt.xlabel('BDT Score Cut')
plt.ylabel('S/sqrt(S+B)')
plt.title(f'Figure of Merit vs. BDT Cut Value\nOptimal Cut = {best_cut:.2f}, FOM = {best_fom:.2f}')
plt.axvline(x=best_cut, color='r', linestyle='--')
plt.grid(True)
plt.savefig("fom_vs_cut.png")

# Apply the optimal cut and perform a final fit
data_df_cut = data_df[data_df['bdt_score'] > best_cut]

In [ ]:

# Function for final fit with optimal cut
def final_fit(df, mass_range=(5300, 5500)):
    # Create histogram of mass
    hist, bin_edges = np.histogram(df['B_M'], bins=40, range=mass_range)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    bin_width = bin_edges[1] - bin_edges[0]
    
    # Define fit functions
    def gaussian(x, amp, mu, sigma):
        return amp * np.exp(-0.5 * ((x - mu) / sigma)**2)
    
    def exponential(x, a, b):
        return a * np.exp(-b * x)
    
    def combined_model(x, amp_gauss, mu_gauss, sigma_gauss, a_exp, b_exp):
        return gaussian(x, amp_gauss, mu_gauss, sigma_gauss) + exponential(x, a_exp, b_exp)
    
    # Initial guesses
    p0 = [100, fit_results['mu'], fit_results['sigma'], 1000, 0.001]
    
    # Fit
    popt, pcov = opt.curve_fit(combined_model, bin_centers, hist, p0=p0)
    
    # Extract parameters
    amp_gauss, mu_gauss, sigma_gauss, a_exp, b_exp = popt
    perr = np.sqrt(np.diag(pcov))
    
    # Calculate signal and background in ±3σ region
    signal_region = (mu_gauss - 3*sigma_gauss, mu_gauss + 3*sigma_gauss)
    
    # Count events in the signal region
    mask_sr = (df['B_M'] >= signal_region[0]) & (df['B_M'] <= signal_region[1])
    events_in_sr = len(df[mask_sr])
    
    # Fit components in signal region
    x_signal_region = np.linspace(signal_region[0], signal_region[1], 1000)
    bin_width_sr = (signal_region[1] - signal_region[0]) / len(x_signal_region)
    
    signal_component = gaussian(x_signal_region, amp_gauss, mu_gauss, sigma_gauss)
    background_component = exponential(x_signal_region, a_exp, b_exp)
    
    # Integrate to get event counts in signal region
    S = np.sum(signal_component) * bin_width_sr
    B = np.sum(background_component) * bin_width_sr
    
    # Calculate S/B ratio
    s_over_b = S / B if B > 0 else float('inf')
    
    # Goodness of fit
    expected = combined_model(bin_centers, *popt)
    chi2_val = np.sum((hist - expected)**2 / expected)
    ndof = len(bin_centers) - len(popt)
    
    # Plot the fit
    plt.figure(figsize=(12, 9))
    plt.hist(df['B_M'], bins=40, range=mass_range, alpha=0.6, label='Data')
    
    x_fit = np.linspace(mass_range[0], mass_range[1], 1000)
    plt.plot(x_fit, combined_model(x_fit, *popt) * bin_width, 'r-', label='Combined Fit')
    plt.plot(x_fit, gaussian(x_fit, amp_gauss, mu_gauss, sigma_gauss) * bin_width, 'g--', label='Signal')
    plt.plot(x_fit, exponential(x_fit, a_exp, b_exp) * bin_width, 'b--', label='Background')
    
    # Highlight signal region
    plt.axvspan(signal_region[0], signal_region[1], alpha=0.3, color='yellow', label='±3σ Region')
    
    plt.xlabel('B_M (MeV)')
    plt.ylabel('Events / ({:.1f} MeV)'.format(bin_width))
    plt.title(r'Fit to $B_s^0 \to D_s^- \pi^+$ Mass Distribution with BDT Cut > {:.2f}'.format(best_cut))
    plt.legend()
    
    plt.text(mass_range[0] + 10, 0.8 * max(hist), 
             f'Signal yield = {S:.1f} ± {perr[0]:.1f}\n'
             f'Background yield = {B:.1f}\n'
             f'$\mu$ = {mu_gauss:.2f} ± {perr[1]:.2f} MeV\n'
             f'$\sigma$ = {sigma_gauss:.2f} ± {perr[2]:.2f} MeV\n'
             f'S/B in ±3σ = {s_over_b:.2f}\n'
             f'$\chi^2$/ndof = {chi2_val:.1f}/{ndof} = {chi2_val/ndof:.2f}')
    
    plt.tight_layout()
    plt.savefig("final_mass_fit.png")
    
    return {
        'mu': mu_gauss,
        'sigma': sigma_gauss,
        'signal': S,
        'background': B,
        'S_over_B': s_over_b,
        'events_in_sr': events_in_sr
    }

In [ ]:
# Perform final fit with optimal cut
final_results = final_fit(data_df_cut)
print("Final fit results after BDT cut:", final_results)
print(f"Signal-to-background ratio in ±3σ region: {final_results['S_over_B']:.2f}")

# Plot BDT score distribution for signal and background
plt.figure(figsize=(10, 6))
plt.hist(data_predictions, bins=50, range=(0, 1), alpha=0.6, label='Data')

# Overlay signal and background distributions from MC
sig_pred = model.predict_proba(signal_df[features])[:, 1]
bkg_pred = model.predict_proba(background_df[features])[:, 1]

plt.hist(sig_pred, bins=50, range=(0, 1), alpha=0.6, label='Signal MC', color='green', 
         weights=np.ones_like(sig_pred)*len(data_predictions)/len(sig_pred))
plt.hist(bkg_pred, bins=50, range=(0, 1), alpha=0.6, label='Background MC', color='red',
         weights=np.ones_like(bkg_pred)*len(data_predictions)/len(bkg_pred))

plt.axvline(x=best_cut, color='r', linestyle='--', label=f'Optimal Cut = {best_cut:.2f}')
plt.xlabel('BDT Score')
plt.ylabel('Events')
plt.title('BDT Score Distribution')
plt.legend()
plt.grid(True)
plt.savefig("bdt_score_distribution.png")

# Summary of results
print("\nSUMMARY OF RESULTS:")
print(f"Initial signal-to-background ratio: {fit_results['S_over_B']:.2f}")
print(f"Optimal BDT cut: {best_cut:.2f}")
print(f"Final signal-to-background ratio: {final_results['S_over_B']:.2f}")
print(f"Improvement factor: {final_results['S_over_B']/fit_results['S_over_B']:.2f}x")